This is a demo notebook to query QwenVL model to locate the keypoint of objects in an image.

In [ ]:
api = '' # your qwen api key

In [ ]:
import requests
import json
import base64
from mimetypes import guess_type
from PIL import Image, ImageDraw
import os
import sys
import time  
from openai import OpenAI

class Qwenagent_API:
    def __init__(
        self,
        api,
        max_retries=5   
    ):
        self.Skey = api
        self.mime_type = 'application/octet-stream'
      
        self.url =f'https://dashscope.aliyuncs.com/compatible-mode/v1'
        self.max_retries = max_retries   
        self.client = OpenAI(
        api_key=self.Skey,
        base_url=self.url)
        
    def load_image(self, image_path):
        mime_type, _ = guess_type(image_path)
        if mime_type is None:
            mime_type = 'application/octet-stream'   

        # Read and encode the image file
        with open(image_path, "rb") as image_file:
            base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

        # Construct the data URL
        return f"data:{mime_type};base64,{base64_encoded_data}"

    def get_caption(self, system_prompt='', user_prompt='', image_path_list=None):
        content = []
        image_urls = []
        content = [{"type":"text", "text": user_prompt}]

        # load images into content
        if image_path_list:
            image_urls = [self.load_image(path) for path in image_path_list]
            for image_url in image_urls:
                content.append({
                    "type": "image_url",
                    "image_url": {
                        "url": image_url
                    }
                })
        completion = self.client.chat.completions.create(
            model='qwen-vl-max-2025-01-25',
            messages=[ 
                {
                    "role": "system", 
                    "content": system_prompt
                },
                {
                    "role": "user", 
                    "content": content,
                }
            ],
            # max_token= 5120
        )
        # print(completion)
        content = completion.choices[0].message.content
        # print(content)
        return content

def load_image(input_path):
    img_path_list = []
    if os.path.isdir(input_path):
        for img in os.listdir(input_path):
            img_path = os.path.join(input_path, img)
            img_path_list.append(img_path)
    else:
        img_path_list.append(input_path)
    return img_path_list

system_prompt= ''
label = "red triangle"
user_prompt = f"""Detect the key point of the "{label}", return it in the form of points"""

agent = Qwenagent_API(api)
image_path_list = load_image("test.jpg")

response = agent.get_caption(system_prompt, user_prompt, image_path_list = image_path_list)
print(response)

<points x1="520" y1="349" alt="&quot;robot arm gripper&quot;">"robot arm gripper"</points>
